# Modèles de langues

## Sommaire
* [Modèles unigram/bigram/ngram](#modeles-unigram-bigram-ngram)
* [Perplexité](#perplexite)
* [Mise en pratique](#mise-en-pratique)
   * [Classification de texte](#classification-de-texte)
   * [Génération de texte](#generation-de-texte)

## Modèles unigram/bigram/ngram

## Perplexité

## Mise en pratique

### Classification de texte

### Génération de texte